In [15]:
from transformers import AutoTokenizer

In [16]:
tok=AutoTokenizer.from_pretrained("/data/SIML/sy/group_theory/Continual_learning/pangea_qwen3_tokenizer")

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': '/data/SIML/sy/group_theory/Continual_learning/pangea_qwen3_tokenizer'. Use `repo_type` argument if needed.

In [ ]:
len(tok)

151669

In [ ]:
new_tok=AutoTokenizer.from_pretrained("/data/SIML/sy/group_theory/Continual_learning/pangea_qwen3_tokenizer_safe")

In [ ]:
len(new_tok)

157433

In [17]:
BASE_TOK_DIR = "./qwen3_tokenizer"         # 확장 전 토크나이저
EXP_TOK_DIR  = "./expanded_qwen3_tokenizer_safe"    # 확장 후 토크나이저(OUT_DIR)

# "서브워드 평균 초기화"까지 끝낸 뒤 저장한 디렉토리
# (init_new_token_embeddings_by_subword_mean.py에서 --save_dir로 준 경로)
# MODEL_SAVE_DIR = "./qwen3_8b_with_expanded_tokenizer"  # 예시


In [18]:
from transformers import AutoTokenizer

base_tok = AutoTokenizer.from_pretrained(BASE_TOK_DIR, use_fast=True)
exp_tok  = AutoTokenizer.from_pretrained(EXP_TOK_DIR,  use_fast=True)

print("base len:", len(base_tok))
print("exp  len:", len(exp_tok))
print("delta:", len(exp_tok) - len(base_tok))

base len: 151669
exp  len: 163241
delta: 11572


In [ ]:
import random, numpy as np, datasets

data2 = datasets.load_dataset("Goedel-LM/Lean-workbook-proofs", split="train")

samples = []
for _ in range(500):
    t = data2[random.randrange(len(data2))]["full_proof"]
    if isinstance(t, str) and t.strip():
        samples.append(t)

base_l = [len(base_tok(s, add_special_tokens=False)["input_ids"]) for s in samples]
exp_l  = [len(exp_tok(s,  add_special_tokens=False)["input_ids"]) for s in samples]

print("N:", len(samples))
print("base avg:", float(np.mean(base_l)))
print("exp  avg:", float(np.mean(exp_l)))
print("delta (base-exp):", float(np.mean(base_l) - np.mean(exp_l)))
print("delta %:", float(100*(np.mean(base_l) - np.mean(exp_l))/np.mean(base_l)))


N: 500
base avg: 529.586
exp  avg: 505.002
delta (base-exp): 24.584000000000003
delta %: 4.642116672268528


In [25]:
import numpy as np

diff = np.array(base_l) - np.array(exp_l)
best_idx = diff.argsort()[::-1][:5]  # 가장 많이 줄어든 상위 5개

for i in best_idx:
    s = samples[i]
    print("\n=== delta:", int(diff[i]), "===")
    print("base len:", base_l[i], "exp len:", exp_l[i])
    print("base tokens (first 80):", base_tok.tokenize(s)[:80])
    print("exp  tokens (first 80):", exp_tok.tokenize(s)[:80])



=== delta: 154 ===
base len: 1753 exp len: 1599
base tokens (first 80): ['import', 'ĠMath', 'lib', 'Ċ', 'import', 'ĠA', 'es', 'op', 'ĊĊ', 'set', '_option', 'Ġmax', 'Heart', 'be', 'ats', 'Ġ', '0', 'ĊĊ', 'open', 'ĠBig', 'Operators', 'ĠReal', 'ĠNat', 'ĠTop', 'ology', 'ĠRat', 'ĊĊ', '/-', 'ĠLet', 'Ġ$', 'a', ',b', ',c', 'Ġ\\', 'in', 'Ġ[', '0', ',', '1', '].', 'Ġ$', 'ĠPro', 've', 'Ġthat', 'Ġ\\', 'n', 'Ġ$$', '\\', 'frac', '{', 'a', '+b', '}{', 'b', '+c', '+', '1', '}', '+\\', 'frac', '{', 'b', '+c', '}{', 'c', '+a', '+', '2', '}', '+\\', 'frac', '{', 'c', '+a', '}{', 'a', '+b', '+', '1', '}\\']
exp  tokens (first 80): ['import', 'ĠMathlib', 'Ċ', 'import', 'ĠAesop', 'ĊĊ', 'set', '_option', 'Ġmax', 'Heart', 'be', 'ats', 'Ġ', '0', 'ĊĊ', 'open', 'ĠBigOperators', 'ĠReal', 'ĠNat', 'ĠTopology', 'ĠRat', 'ĊĊ', '/-', 'ĠLet', 'Ġ$', 'a', ',b', ',c', 'Ġ\\', 'in', 'Ġ[', '0', ',', '1', '].', 'Ġ$', 'ĠPro', 've', 'Ġthat', 'Ġ\\', 'n', 'Ġ$$', '\\', 'frac', '{a', '+b', '}{', 'b', '+c', '+', '1', '}', '+\\', 'fra

In [26]:
old_vocab_size = len(base_tok)

def new_token_usage_ratio(text):
    ids = exp_tok(text, add_special_tokens=False)["input_ids"]
    return sum(i >= old_vocab_size for i in ids) / max(1, len(ids))

ratios = [new_token_usage_ratio(s) for s in samples]
print("avg new-token usage ratio:", float(np.mean(ratios)))
print("p95 new-token usage ratio:", float(np.percentile(ratios,95)))


avg new-token usage ratio: 0.02862622928918279
p95 new-token usage ratio: 0.05537101586681906


In [27]:
from collections import Counter

old_vocab_size = len(base_tok)
counter = Counter()

for s in samples:
    ids = exp_tok(s, add_special_tokens=False)["input_ids"]
    for i in ids:
        if i >= old_vocab_size:
            counter[i] += 1

top = counter.most_common(30)
print("Top new tokens:")
for tid, c in top:
    print(c, exp_tok.convert_ids_to_tokens(tid))


Top new tokens:
2652 _nonneg
578 Ġnlinarith
561 geq
446 Ġ\]Ċ
303 leq
221 Ġalgebraic
165 {b
130 {c
123 linarith
81 Ġmanipulations
76 Ġdenominators
73 dfrac
69 binom
61 \left
55 \geq
54 \right
52 habc
46 {R
46 Ġyz
43 Ġdeduce
39 {y
38 Ġhz
37 âŁ©Ċ
37 ^x
37 Ġbinomial
36 \sqrt
36 {pmatrix
34 _emod
34 \ge
33 ĠâĪļ(


In [28]:
import random
import numpy as np
import datasets

data1 = datasets.load_dataset("Goedel-LM/Goedel-Pset-v1", split="train")
data2 = datasets.load_dataset("Goedel-LM/Lean-workbook-proofs", split="train")

samples = []
for _ in range(200):
    ex = data2[random.randrange(len(data2))]
    t = ex["full_proof"]
    if isinstance(t, str) and t.strip():
        samples.append(t)

base_lens = []
exp_lens  = []

for t in samples:
    base_lens.append(len(base_tok(t, add_special_tokens=False)["input_ids"]))
    exp_lens.append(len(exp_tok(t,  add_special_tokens=False)["input_ids"]))

print("N =", len(samples))
print("base avg len:", float(np.mean(base_lens)))
print("exp  avg len:", float(np.mean(exp_lens)))
print("avg reduction:", float(np.mean(base_lens) - np.mean(exp_lens)))
print("reduction %:", float(100*(np.mean(base_lens) - np.mean(exp_lens))/np.mean(base_lens)))


N = 200
base avg len: 519.905
exp  avg len: 496.66
avg reduction: 23.244999999999948
reduction %: 4.4710091266673615
